In [9]:
from utils import (read_video, 
                   save_video,
                   measure_distance,
                   draw_player_stats,
                   convert_pixel_distance_to_meters
                   )
import constants
from trackers import PlayerTracker, BallTracker
from court_line_detector import CourtLineDetector
from mini_court import MiniCourt
import cv2
import pandas as pd
from copy import deepcopy


def main():
    # Read Video
    input_video_path = "input_videos/input_video-2.mp4"
    video_frames = read_video(input_video_path)

    # Detect Players and Ball
    player_tracker = PlayerTracker(model_path='yolov8x')
    ball_tracker = BallTracker(model_path='models/yolo5_last.pt')

    player_detections = player_tracker.detect_frames(video_frames,
                                                     read_from_stub=True,
                                                     stub_path="tracker_stubs/player_detections.pkl"
                                                     )
    ball_detections = ball_tracker.detect_frames(video_frames,
                                                     read_from_stub=True,
                                                     stub_path="tracker_stubs/ball_detections.pkl"
                                                     )
    ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)

    # Court Line Detector model
    court_model_path = "models/keypoints_model.pth"
    court_line_detector = CourtLineDetector(court_model_path)
    court_keypoints = court_line_detector.predict(video_frames[0])

    # choose players
    player_detections = player_tracker.choose_and_filter_players(court_keypoints, player_detections)

    # MiniCourt
    mini_court = MiniCourt(video_frames[0])

    # Detect ball shots
    ball_shot_frames = ball_tracker.get_ball_shot_frames(ball_detections)

    # Convert positions to mini court positions
    player_mini_court_detections, ball_mini_court_detections = mini_court.convert_bounding_boxes_to_mini_court_coordinates(
        player_detections, 
        ball_detections,
        court_keypoints
    )

    player_stats_data = [{
        'frame_num': 0,
        'player_1_number_of_shots': 0,
        'player_1_total_shot_speed': 0,
        'player_1_last_shot_speed': 0,
        'player_1_total_player_speed': 0,
        'player_1_last_player_speed': 0,

        'player_2_number_of_shots': 0,
        'player_2_total_shot_speed': 0,
        'player_2_last_shot_speed': 0,
        'player_2_total_player_speed': 0,
        'player_2_last_player_speed': 0,
    }]

    for ball_shot_ind in range(len(ball_shot_frames) - 1):
        start_frame = ball_shot_frames[ball_shot_ind]
        end_frame = ball_shot_frames[ball_shot_ind + 1]
        ball_shot_time_in_seconds = (end_frame - start_frame) / 24  # Assuming 24 FPS

        # Distance and Speed for Ball
        distance_covered_by_ball_pixels = measure_distance(
            ball_mini_court_detections[start_frame][1],
            ball_mini_court_detections[end_frame][1]
        )
        distance_covered_by_ball_meters = convert_pixel_distance_to_meters(
            distance_covered_by_ball_pixels,
            constants.DOUBLE_LINE_WIDTH,
            mini_court.get_width_of_mini_court()
        )
        speed_of_ball_shot = distance_covered_by_ball_meters / ball_shot_time_in_seconds * 3.6

        # Determine who shot the ball
        player_positions = player_mini_court_detections[start_frame]
        player_shot_ball = min(
            player_positions.keys(), 
            key=lambda player_id: measure_distance(
                player_positions[player_id],
                ball_mini_court_detections[start_frame][1]
            )
        )

        # Opponent movement (safe access)
        opponent_player_id = 1 if player_shot_ball == 2 else 2
        
        # Directly access the list for the start_frame and end_frame positions
        pos_start = player_mini_court_detections[start_frame].get(opponent_player_id, (0, 0))
        pos_end = player_mini_court_detections[end_frame].get(opponent_player_id, (0, 0))
        
        if pos_start == (0, 0) or pos_end == (0, 0):
            distance_covered_by_opponent_pixels = 0
        else:
            distance_covered_by_opponent_pixels = measure_distance(pos_start, pos_end)
        
        distance_covered_by_opponent_meters = convert_pixel_distance_to_meters(
            distance_covered_by_opponent_pixels,
            constants.DOUBLE_LINE_WIDTH,
            mini_court.get_width_of_mini_court()
        )
        speed_of_opponent = distance_covered_by_opponent_meters / ball_shot_time_in_seconds * 3.6


        # Update player stats
        current_player_stats = deepcopy(player_stats_data[-1])
        current_player_stats['frame_num'] = start_frame
        current_player_stats[f'player_{player_shot_ball}_number_of_shots'] += 1
        current_player_stats[f'player_{player_shot_ball}_total_shot_speed'] += speed_of_ball_shot
        current_player_stats[f'player_{player_shot_ball}_last_shot_speed'] = speed_of_ball_shot
        current_player_stats[f'player_{opponent_player_id}_total_player_speed'] += speed_of_opponent
        current_player_stats[f'player_{opponent_player_id}_last_player_speed'] = speed_of_opponent

        player_stats_data.append(current_player_stats)

    player_stats_data_df = pd.DataFrame(player_stats_data)
    frames_df = pd.DataFrame({'frame_num': list(range(len(video_frames)))})
    player_stats_data_df = pd.merge(frames_df, player_stats_data_df, on='frame_num', how='left')
    player_stats_data_df = player_stats_data_df.ffill()

    player_stats_data_df['player_1_average_shot_speed'] = player_stats_data_df['player_1_total_shot_speed'] / player_stats_data_df['player_1_number_of_shots']
    player_stats_data_df['player_2_average_shot_speed'] = player_stats_data_df['player_2_total_shot_speed'] / player_stats_data_df['player_2_number_of_shots']
    player_stats_data_df['player_1_average_player_speed'] = player_stats_data_df['player_1_total_player_speed'] / player_stats_data_df['player_2_number_of_shots']
    player_stats_data_df['player_2_average_player_speed'] = player_stats_data_df['player_2_total_player_speed'] / player_stats_data_df['player_1_number_of_shots']

    # Draw output
    ## Draw Player Bounding Boxes
    output_video_frames = player_tracker.draw_bboxes(video_frames, player_detections)
    output_video_frames = ball_tracker.draw_bboxes(output_video_frames, ball_detections)

    ## Draw court Keypoints
    output_video_frames = court_line_detector.draw_keypoints_on_video(output_video_frames, court_keypoints)

    # Draw Mini Court
    output_video_frames = mini_court.draw_mini_court(output_video_frames)
    output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames, player_mini_court_detections)
    output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames, ball_mini_court_detections, color=(0, 255, 255))

    # Draw Player Stats
    output_video_frames = draw_player_stats(output_video_frames, player_stats_data_df)

    ## Draw frame number on top left corner
    for i, frame in enumerate(output_video_frames):
        cv2.putText(frame, f"Frame: {i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    save_video(output_video_frames, "output_videos/output_video03.avi")


if __name__ == "__main__":
    main()


C:\Users\sayaseng\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sayaseng\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[MiniCourt] Initializing...
[MiniCourt] Setting background box...
[MiniCourt] Background box: (340, 50) to (590, 550)
[MiniCourt] Setting mini court position...
[MiniCourt] Court: (360, 70) to (570, 530)
[MiniCourt] Setting court key points...
[MiniCourt] Court key points set.
[MiniCourt] Setting court lines...
[MiniCourt] Court lines set.
[MiniCourt] Initialization complete.
[MiniCourt] Converting bounding boxes...
[MiniCourt] Frame 0
[MiniCourt] 14.950354451639923 meters -> 286.19639333130203 pixels
[MiniCourt] 0.7418421994654185 meters -> 14.20117246014019 pixels
[MiniCourt] Mapped position: (286.19639333130203, 14.20117246014019)
[MiniCourt] 1.122715834126143 meters -> 21.492281236690065 pixels
[MiniCourt] 0.0701636202279192 meters -> 1.3431504328042871 pixels
[MiniCourt] Mapped position: (21.492281236690065, 1.3431504328042871)
[MiniCourt] Frame 1
[MiniCourt] 15.013177128705454 meters -> 287.399015225902 pixels
[MiniCourt] 0.7516999417689368 meters -> 14.389880380262234 pixels
[Mi

C:\Users\sayaseng\Downloads\HackathonProject\trackers\ball_tracker.py:50: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ball_positions['ball_hit'].iloc[i] = 1
C:\Users\sayaseng\Downloads\HackathonProject\trackers\ball_tracker.py:50: Setti

[MiniCourt] Drawing court...
[MiniCourt] Frame 16
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 17
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 18
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 19
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 20
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 21
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 22
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 23
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 24
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 25
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 26
[MiniCourt] Drawing background...
[MiniCourt] Drawing court...
[MiniCourt] Frame 27
[MiniCourt] Drawing backgr